In [86]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import numpy as np
import plotly.express as px

In [87]:
state_abbreviations = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming', 
    'Washington, DC': 'MD',
    'Long Island-Queens': 'NY', 
    'Township of Lawrence': 'NJ',
    'Manhattan': 'NY',
    'New York State': 'NY',
    'Texhoma': "OK", 
    "United States": "WA"
}
reverse_abbrev = {value:key for (key, value) in state_abbreviations.items()}

In [88]:
job_postings = pd.read_csv("clean_job_postings_w_salary.csv").drop(columns = ['Unnamed: 0'])

In [89]:
job_postings['salary estimate'].apply(lambda x: x.replace("(est.)", '').strip()[-3:]).unique()

array(['/yr', '/hr', '/mo'], dtype=object)

In [90]:
def clean_estimate(estimate):
    estimate = estimate.replace("(est.)", '').replace("$", '').strip()
    suffix = estimate[-4:]
    estimate_int = float(estimate[:-3])
    if "/mo" in suffix:
        return 12 * estimate_int
    elif "/hr" in suffix:
        return estimate_int * 2000
    else:
        return estimate_int

In [91]:
job_postings['salary estimate'] = job_postings['salary estimate'].apply(clean_estimate)

In [92]:
job_postings.groupby("location")['salary estimate'].mean()

location
Abbott Park, IL       102372.000000
Alameda, CA           160000.000000
Albany, NY             68617.000000
Alexandria, VA        112443.857143
Alpharetta, GA        140500.000000
                          ...      
Wilmington, MA        114397.000000
Wisconsin             125350.000000
Woodbridge, NJ         98694.000000
Woodcliff Lake, NJ     81136.000000
Youngsville, LA        77299.000000
Name: salary estimate, Length: 240, dtype: float64

In [93]:
def create_state_col(location):
    exclusion_lst = ['United States', 'San Juan, PR']
    if location not in exclusion_lst:
        if location[-2:] in state_abbreviations:
            return location[-2:]
        elif location in state_abbreviations:
            return state_abbreviations[location]
        elif location in reverse_abbrev:
            return reverse_abbrev[location]
    return "None"

In [94]:
job_postings['state'] = job_postings['location'].apply(create_state_col)

In [95]:
cost_of_living = pd.read_csv("data/cost_of_living_us.csv")
cost_of_living = cost_of_living[cost_of_living.family_member_count=='1p0c']
#cost_of_living.sort_values('total_cost', ascending=False)

In [96]:
cost_of_living.total_cost -= cost_of_living.taxes
cost_of_living = cost_of_living.groupby('state').agg(
    average_total_cost=('total_cost', 'mean'),
    median_total_cost=('total_cost', 'median'),
).reset_index()

In [97]:
agg_data = job_postings.groupby(['job_category', 'state']).agg(
    average_salary=('salary', 'mean'),
    median_salary=('salary', 'median'),
    observations=('salary', 'count')
).reset_index()

# Map state abbreviations to full state names
agg_data['Full_state'] = agg_data['state'].map(state_abbreviations)

# Prepare hovertext
agg_data['hovertext'] = agg_data['Full_state'] + '<br>' + \
                        'Average Salary: ' + agg_data['average_salary'].round(2).astype(str) + '<br>' + \
                        'Observations: ' + agg_data['observations'].astype(str)


In [98]:
remote_jobs = agg_data[agg_data.state=="None"]
remote_jobs.state = 'Remote'
remote_jobs

/var/folders/sx/q8bq3_v13q38gg5c4y_k59tm0000gn/T/ipykernel_14460/2175974504.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,job_category,state,average_salary,median_salary,observations,Full_state,hovertext
23,data analyst,Remote,97767.710526,100000.0,38,NaN,NaN
51,data engineer,Remote,126743.233333,126150.0,48,NaN,NaN
84,data scientist,Remote,129861.907692,130000.0,65,NaN,NaN
106,machine learning engineer,Remote,148598.222222,153619.0,9,NaN,NaN
121,other,Remote,101242.285714,105000.0,7,NaN,NaN


In [99]:
df = pd.merge(agg_data, cost_of_living, on='state', how='inner')
df['avg_salary_after_COL'] = df.average_salary - df.average_total_cost
df['median_salary_after_COL'] = df.median_salary - df.median_total_cost

df.shape

(119, 11)

In [100]:
df = df.append(remote_jobs, ignore_index=True)

/var/folders/sx/q8bq3_v13q38gg5c4y_k59tm0000gn/T/ipykernel_14460/682411431.py:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [101]:
df.to_csv('data/salary_cost_of_living.csv')

In [76]:
df = pd.read_csv("data/salary_cost_of_living.csv")

In [102]:
fig = go.Figure(go.Choropleth(
    locations=df['state'],
    z=df['average_salary'],
    locationmode='USA-states',
    colorbar_title='Average Salary',
    hovertext=df['hovertext']
))

# Update the layout
fig.update_layout(
    title='Average Salary by State in the USA',
    geo=dict(scope='usa')
)

# Show the figure
fig.show()

In [103]:
def create_salary_map(df):
    size_scale = 10 

    blue_to_red = [
        [0, 'rgb(0, 0, 255)'],
        [1, 'rgb(255, 0, 0)'] 
    ]

    fig = make_subplots(
        rows=1, cols=2,
        column_widths=[0.8, 0.2],
        specs=[[{'type': 'scattergeo'}, {'type': 'bar'}]]
    )

    fig.add_trace(
        go.Scattergeo(
            locationmode='USA-states',
            locations=df['state'], 
            text=df['hovertext'],
            marker=dict(
                size=df['observations'] * size_scale, 
                color=df['average_salary'],
                colorscale=blue_to_red,  
                colorbar_title='Average Salary',
                line_color='rgb(40,40,40)',
                line_width=0.5,
                sizemode='area',
                cmin=np.min(df['average_salary']),
                cmax=np.max(df['average_salary']),
                colorbar=dict(x=-0.1)
            ),
            hoverinfo='text'
        ),
        row=1, col=1
    )

    remote_jobs_index = df['state'] == 'Remote'
    mean_salary_remote = df['average_salary'][remote_jobs_index].mean()

    number_of_remote_jobs = df['observations'][remote_jobs_index].sum()

    bar_color = (mean_salary_remote - np.min(df['average_salary'])) / (np.max(df['average_salary']) - np.min(df['average_salary']))
    print(bar_color)
    fig.add_trace(
        go.Bar(
            x=['Remote Jobs'],
            y=[number_of_remote_jobs],
            marker=dict(color=bar_color),
            text=f'Average Salary: ${mean_salary_remote:,.2f}',
            hovertext=f'Average Salary: ${mean_salary_remote:,.2f}, based on {number_of_remote_jobs} jobs', 
            hoverinfo='text',
            textposition='auto'
        ),
        row=1, col=2
    )

    fig.update_layout(
        title_text='Average Salary by State in the USA and Remote Jobs',
        geo=dict(
            scope='usa',
            projection_type='albers usa',
            showland=True,
            landcolor='rgb(217, 217, 217)',
            domain=dict(x=[0, 0.75]),  
        ),
        showlegend=False
    )

    fig.update_layout(
        barmode='stack',
        annotations=[dict(
            x=1.25,
            y=number_of_remote_jobs,
            text='Remote Jobs',
            showarrow=False,
            xref='paper',
            yref='y',
            font=dict(size=16)
        )],
        yaxis2=dict(showticklabels=False)
    )

    fig.update_layout(
        width=1000,
        height=600
    )

    fig.show()


In [105]:
df

,job_category,state,average_salary,median_salary,observations,Full_state,hovertext,average_total_cost,median_total_cost,avg_salary_after_COL,median_salary_after_COL
0,data analyst,AL,53877.000000,53877.0,1,Alabama,Alabama<br>Average Salary: 53877.0<br>Observat...,29860.222750,29315.44344,24016.777250,24561.55656
1,data analyst,AR,74844.000000,74844.0,1,Arkansas,Arkansas<br>Average Salary: 74844.0<br>Observa...,28304.626782,28087.95564,46539.373218,46756.04436
2,data scientist,AR,54281.000000,54281.0,1,Arkansas,Arkansas<br>Average Salary: 54281.0<br>Observa...,28304.626782,28087.95564,25976.373218,26193.04436
3,data analyst,AZ,68167.600000,64075.0,3,Arizona,Arizona<br>Average Salary: 68167.6<br>Observat...,32551.364504,32436.23340,35616.235496,31638.76660
4,data engineer,AZ,88617.500000,88617.5,2,Arizona,Arizona<br>Average Salary: 88617.5<br>Observat...,32551.364504,32436.23340,56066.135496,56181.26660
...,...,...,...,...,...,...,...,...,...,...,...
119,data analyst,Remote,97767.710526,100000.0,38,NaN,NaN,NaN,NaN,NaN,NaN
120,data engineer,Remote,126743.233333,126150.0,48,NaN,NaN,NaN,NaN,NaN,NaN
121,data scientist,Remote,129861.907692,130000.0,65,NaN,NaN,NaN,NaN,NaN,NaN
122,machine learning engineer,Remote,148598.222222,153619.0,9,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
create_salary_map(df)

0.4611928907319927


In [80]:
df

,Unnamed: 0,state,average_salary,median_salary,observations,Full_state,hovertext,average_total_cost,median_total_cost,avg_salary_after_COL,median_salary_after_COL
0,0,AL,53877.000000,53877.0,1,Alabama,Alabama<br>Average Salary: 53877.0<br>Observat...,68967.570020,73026.2868,-15090.570020,-19149.2868
1,1,AR,64562.500000,64562.5,2,Arkansas,Arkansas<br>Average Salary: 64562.5<br>Observa...,61491.891851,64790.3994,3070.608149,-227.8994
2,2,AZ,76347.560000,64075.0,5,Arizona,Arizona<br>Average Salary: 76347.56<br>Observa...,75985.939376,76835.3088,361.620624,-12760.3088
3,3,CA,128569.560000,128320.0,100,California,California<br>Average Salary: 128569.56<br>Obs...,93179.163060,89793.1464,35390.396940,38526.8536
4,4,CO,108270.300000,100213.5,20,Colorado,Colorado<br>Average Salary: 108270.3<br>Observ...,77595.782964,77275.3794,30674.517036,22938.1206
5,5,CT,128590.000000,135000.0,4,Connecticut,Connecticut<br>Average Salary: 128590.0<br>Obs...,90013.504620,92519.2326,38576.495380,42480.7674
6,6,DE,78724.666667,78000.0,3,Delaware,Delaware<br>Average Salary: 78724.67<br>Observ...,81505.517560,83640.7590,-2780.850893,-5640.7590
7,7,FL,84523.210526,89347.0,19,Florida,Florida<br>Average Salary: 84523.21<br>Observa...,71521.722981,71941.6962,13001.487545,17405.3038
8,8,GA,101447.375000,105718.0,8,Georgia,Georgia<br>Average Salary: 101447.38<br>Observ...,64898.336594,66149.1096,36549.038406,39568.8904
9,9,IA,83582.200000,86265.0,5,Iowa,Iowa<br>Average Salary: 83582.2<br>Observation...,66089.039308,69293.9736,17493.160692,16971.0264


In [84]:
df = df.sort_values(by='avg_salary_after_COL', ascending=False)
df=df[df.state!='Remote']
df=df[df.observations>=5]
fig = px.bar(df,
             x='state',
             y='avg_salary_after_COL',
             title='Pre-Tax Average Income After Cost of Living Expenses in US States',
             labels={'avg_salary_after_COL': 'Average Salary', 'Unfiltered_State': 'State'},
             color='avg_salary_after_COL')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [82]:
df = df.sort_values(by='median_salary_after_COL', ascending=False)
df=df[df.state!='Remote']
df=df[df.observations>=5]
fig = px.bar(df,
             x='state',
             y='median_salary_after_COL',
             title='Median Salary by State in the USA',
             labels={'median_salary_after_COL': 'Average Salary', 'Unfiltered_State': 'State'},
             color='median_salary_after_COL')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [13]:
df

,Unnamed: 0,state,average_salary,median_salary,observations,Full_state,hovertext,average_total_cost,median_total_cost,avg_salary_after_COL,median_salary_after_COL
24,24,NM,140000.000000,140000.0,2,New Mexico,New Mexico<br>Average Salary: 140000.0<br>Obse...,67687.562087,70199.2530,72312.437913,69800.7470
37,37,WA,140739.909091,129552.5,22,Washington,Washington<br>Average Salary: 140739.91<br>Obs...,75648.910151,76483.5618,65090.998940,53068.9382
5,5,CT,128590.000000,135000.0,4,Connecticut,Connecticut<br>Average Salary: 128590.0<br>Obs...,90013.504620,92519.2326,38576.495380,42480.7674
28,28,OK,113140.750000,117000.0,4,Oklahoma,Oklahoma<br>Average Salary: 113140.75<br>Obser...,71110.675373,74993.8914,42030.074627,42006.1086
8,8,GA,101447.375000,105718.0,8,Georgia,Georgia<br>Average Salary: 101447.38<br>Observ...,64898.336594,66149.1096,36549.038406,39568.8904
3,3,CA,128569.560000,128320.0,100,California,California<br>Average Salary: 128569.56<br>Obs...,93179.163060,89793.1464,35390.396940,38526.8536
22,22,NE,117500.000000,117500.0,1,Nebraska,Nebraska<br>Average Salary: 117500.0<br>Observ...,77684.601942,79078.6320,39815.398058,38421.3680
32,32,SC,92347.250000,99750.0,4,South Carolina,South Carolina<br>Average Salary: 92347.25<br>...,62104.869806,62434.0752,30242.380194,37315.9248
34,34,TX,98756.184211,99984.5,38,Texas,Texas<br>Average Salary: 98756.18<br>Observati...,64795.398352,66925.7076,33960.785859,33058.7924
21,21,NC,103204.887500,104000.0,16,North Carolina,North Carolina<br>Average Salary: 103204.89<br...,70013.586953,71188.1892,33191.300547,32811.8108
